# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cityDataFrame = pd.read_csv(r"C:\Users\mengc\Desktop\06-Python-APIs\Homework\Instructions\output_data\cities.csv")
cityDataFrame.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vaini,-21.20,-175.20,75.20,94,75,5.82,TO,1594109659
1,1,kruisfontein,-34.00,24.73,57.69,61,99,25.61,ZA,1594109729
2,2,guerrero negro,27.98,-114.06,67.91,79,0,6.29,MX,1594109786
3,3,bom jesus,-9.07,-44.36,64.17,67,0,1.07,BR,1594110108
4,4,ponta do sol,32.67,-17.10,70.00,77,20,2.24,PT,1594109738


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=g_key)

In [4]:
place = cityDataFrame[["Lat", "Lng"]]
humidity = cityDataFrame["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(place, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
criteriaDataFrame = cityDataFrame.loc[(cityDataFrame["Max Temp"] < 80) & (cityDataFrame["Max Temp"] > 70) \
                                    & (cityDataFrame["Wind Speed"] < 10) \
                                    & (cityDataFrame["Cloudiness"] == 0)].dropna()
criteriaDataFrame

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
100,100,waddan,29.16,16.14,79.32,42,0,8.95,LY,1594110128
107,107,derzhavinsk,51.10,66.32,78.91,44,0,3.87,KZ,1594110130
114,114,hun,29.13,15.95,79.52,41,0,8.01,LY,1594110131
162,162,puerto del rosario,28.50,-13.86,78.80,69,0,3.36,ES,1594110140
301,301,tyup,42.73,78.36,71.82,25,0,8.59,KG,1594110170
321,321,richards bay,-28.78,32.04,72.36,22,0,4.76,ZA,1594110174
473,473,mpulungu,-8.76,31.11,79.23,33,0,3.65,ZM,1594110207


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = criteriaDataFrame[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
100,waddan,LY,29.16,16.14,
107,derzhavinsk,KZ,51.10,66.32,
114,hun,LY,29.13,15.95,
162,puerto del rosario,ES,28.50,-13.86,
301,tyup,KG,42.73,78.36,
321,richards bay,ZA,-28.78,32.04,
473,mpulungu,ZM,-8.76,31.11,


In [13]:
#params
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
100,waddan,LY,29.16,16.14,
107,derzhavinsk,KZ,51.10,66.32,
114,hun,LY,29.13,15.95,
162,puerto del rosario,ES,28.50,-13.86,
301,tyup,KG,42.73,78.36,
321,richards bay,ZA,-28.78,32.04,
473,mpulungu,ZM,-8.76,31.11,


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
cityDataFrame = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [cityDataFrame.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))